In [1]:
import os
import sys
# Note: you might have to change the path to where you put the linear_alg.py source file
sys.path.append(os.getcwd() + "/LinearAlg")

In [2]:
from linear_alg import *

In [3]:
"""
Example of row reducing matrices
"""
A = Mat([[1,2,3,4],[5,6,7,8],[9,10,11,12]])
A.print(name="A")
print()
A.to_echelon_form()
A.round(8) # Get rid of ugly floating point errors
A.print(name="A in echelon form")
print()
A.to_reduced_echelon_form(already_echelon_form=True)
A.round(8)
A.print(name="A in reduced echelon form")
print()

"""
If you want to avoid precision errors, matrices and vectors (mostly) work with fractions
"""
from fractions import Fraction
B = Mat([[Fraction(1,2),Fraction(3,1),Fraction(5,6)],[Fraction(2,3),Fraction(4,5),Fraction(2,5)]])
B.print(name="B")
print()
B.to_echelon_form()
B.print(name="B in echelon form")
print()
B.to_reduced_echelon_form(already_echelon_form=True)
B.print(name="B in reduced echelon form")

A: Matrix 3x4:
 1,   2,   3,   4,  
 5,   6,   7,   8,  
 9,   10,  11,  12, 

A in echelon form: Matrix 3x4:
 1.0,         1.11111111,  1.22222222,  1.33333333, 
 0.0,         1.0,         2.0,         3.0,        
 0.0,         0.0,         0.0,         0.0,        

A in reduced echelon form: Matrix 3x4:
 1.0,  0.0, -1.0, -2.0, 
 0.0,  1.0,  2.0,  3.0, 
 0.0,  0.0,  0.0,  0.0, 

B: Matrix 2x3:
 1/2,  3,    5/6, 
 2/3,  4/5,  2/5, 

B in echelon form: Matrix 2x3:
 1,    6/5,  3/5, 
 0,    1,    2/9, 

B in reduced echelon form: Matrix 2x3:
 1,    0,    1/3, 
 0,    1,    2/9, 


In [4]:
"""
Finding matrix inverses
"""
A = Mat([[5,4,6],[3,1,7],[3,6,7]])
A.print(name="A")
print("Determinant:", A.determinant())
print()

A_inv = A.inverse()
A_inv.print(name="A inverse")

A: Matrix 3x3:
 5,  4,  6, 
 3,  1,  7, 
 3,  6,  7, 
Determinant: -85

A inverse: Matrix 3x3:
 0.411764705882353,   -0.09411764705882354, -0.25882352941176473, 
 0.0,                 -0.2,                  0.2,                 
-0.1764705882352941,   0.21176470588235294,  0.08235294117647057, 


In [5]:
"""
Example of finding eigenvalues
"""
A = Mat([[1,2,3],[4,5,6],[7,8,9]])
A.print(name="A")
print()

λ = Var("λ")
A_minus_lambda = A - λ*Mat.identity(3)
A_minus_lambda.print(name="A-λI")
print()

characteristic_polynomial = A_minus_lambda.determinant()
print("characteristic_polynomial:", characteristic_polynomial)

# My equations class is still very rudimentary; currently, it can only do basic operations with polynomials

A: Matrix 3x3:
 1,  2,  3, 
 4,  5,  6, 
 7,  8,  9, 

A-λI: Matrix 3x3:
 (-1 * λ) + 1,  2,             3,            
 4,             (-1 * λ) + 5,  6,            
 7,             8,             (-1 * λ) + 9, 

characteristic_polynomial: (18 * λ) + (15 * λ^2) + (-1 * λ^3)


In [6]:
"""
Example of normalizing a basis
"""
basis = [
    Vec([1,1,1,1]),
    Vec([0,1,1,1]),
    Vec([0,0,1,1])
]
ortho_basis = orthogonalize(basis)
orthonormal_basis = [v.normalize() for v in ortho_basis]
print("ortho_basis:\n", ortho_basis)
print("\northonormal_basis:\n",orthonormal_basis)

ortho_basis:
 [Vector([1, 1, 1, 1]), Vector([-0.75, 0.25, 0.25, 0.25]), Vector([0.0, -0.6666666666666666, 0.33333333333333337, 0.33333333333333337])]

orthonormal_basis:
 [Vector([0.5, 0.5, 0.5, 0.5]), Vector([-0.8660254037844388, 0.2886751345948129, 0.2886751345948129, 0.2886751345948129]), Vector([0, -0.8164965809277259, 0.408248290463863, 0.408248290463863])]


In [7]:
import time
import random

def project(x, constants, b, damping=1):
    # Project x onto the equation constants[0]*x + ... + constants[n]*x = b
    return ((b-x.dot(constants)) / constants.dot(constants)) * constants * damping + x

n = 6
random.seed(12345)
def rand():
    return random.randrange(-20,20)

m = []
for i in range(n):
    m.append([rand() for _ in range(n)])
m = Mat(m)
m.print()
b = Vec([rand() for _ in range(n)])

m_inv = m.inverse()
x = m_inv*b
print("True x:", x)
x = Vec([0]*n)
i = 0
EPSILON = 0.01
nr_projections = 0
t = time.time()
while abs(m*x-b).magnitude() > EPSILON and nr_projections <= 10e5:
    for i in range(len(m.rows)):
        x = project(x, m.rows[i], b[i], damping=1)
        nr_projections += 1
# x.round(3)
t = time.time()-t
print("final:", x)
print("nr_projections:", nr_projections)
print("time:", t)

Matrix 6x6:
 6,  -20, -1,   3,  -8,  -3,  
 16,  7,  -10,  3,  -13,  7,  
-4,   15, -9,   19,  15, -9,  
 2,  -15,  13,  6,   17,  12, 
-10, -11, -7,  -16, -8,   1,  
 0,  -19,  9,   1,  -19,  12, 
True x: [0.1180812463120422, -0.32896437311974613, 0.4845110614546759, -1.2675505189742398, 0.12442787080298326, -1.3316035481780275]
final: [0.11776975400350566, -0.32893626688211464, 0.48413084041219184, -1.2671756829990133, 0.12443877450969237, -1.3311477587902991]
nr_projections: 222
time: 0.0038309097290039062


In [8]:
r1 = Vec([1,1,0])
r2 = Vec([2,1,1])
r3 = Vec([4,1,2])
r4 = Vec([5,1,3])
m = Mat([r1,r2,r3,r4])
m.print()
r2 -= 2*r1
print(r2)
r3 -= 4*r1
print(r3)
r4 -= 5*r1
print(r4)
r3 -= 3*r2
print(r3)
r4 -= 4*r2
print(r4)
Mat([r1,r2,r3,r4]).print()
m.to_reduced_echelon_form()
m.print()

Matrix 4x3:
 1,  1,  0, 
 2,  1,  1, 
 4,  1,  2, 
 5,  1,  3, 
[0, -1, 1]
[0, -3, 2]
[0, -4, 3]
[0, 0, -1]
[0, 0, -1]
Matrix 4x3:
 1,  1,  0, 
 0, -1,  1, 
 0,  0, -1, 
 0,  0, -1, 
Matrix 4x3:
 1,    0.0,  0.0, 
 0,    1,    0.0, 
 0,    0,    1,   
 0,    0.0,  0.0, 


In [9]:
A = Mat([[1,1],[2,1],[4,1],[5,1]])
b = Vec([0,1,2,3])
A.print()
A_t = A.transpose()
A_t.print()
(A_t*A).print()
print(A_t*b)
print(46*4)

Matrix 4x2:
 1,  1, 
 2,  1, 
 4,  1, 
 5,  1, 
Matrix 2x4:
 1,  2,  4,  5, 
 1,  1,  1,  1, 
Matrix 2x2:
 46,  12, 
 12,  4,  
[25, 6]
184


In [10]:
from fractions import Fraction

In [11]:
print(Fraction(28,40))
print(Fraction(-24,40))

7/10
-3/5


In [12]:
M = Mat([[0.9871,0.0027],[0.0129,0.9973]])
M.print()
print(0.9871+.0129)
print(0.0027+0.9973)
x0 = Vec([0.125, 0.875])
m8 = (M*M*M*M*M*M*M*M)
m8.round(4)
m8.print()
print(m8*x0)
λ = Var("λ")
Mml = (M-λ*Mat.identity(2))
Mml.print()
print(Mml.determinant())
print((λ-0.9844)*(λ-1))

nm = M-Mat.identity(2)
nm.print()

v1 = Vec([1,-1])
print(M*v1)
v2 = Vec([0.0027,0.0129])
print(M*v2)

Matrix 2x2:
 0.9871,  0.0027, 
 0.0129,  0.9973, 
1.0
1.0
Matrix 2x2:
 0.9023,  0.0205, 
 0.0977,  0.9795, 
[0.130725, 0.869275]
Matrix 2x2:
 (-1 * λ) + 0.9871,  0.0027,            
 0.0129,             (-1 * λ) + 0.9973, 
(-1.9844 * λ) + (λ^2) + 0.9843999999999999
(-1.9844 * λ) + (λ^2) + 0.9844
Matrix 2x2:
-0.012900000000000023,   0.0027,                
 0.0129,                -0.0027000000000000357, 
[0.9843999999999999, -0.9843999999999999]
[0.0027, 0.012899999999999998]


In [13]:
r1 = Vec([1,0.0027,0.125])
r2 = Vec([-1,0.0129,0.875])
m = Mat([r1,r2])
m.print()
r2 += r1
print(r2)
r2 /= 0.0156
print(r2)
r1 -= 0.0027*r2
print(r1)
Mat([r1,r2]).print()

m.to_reduced_echelon_form()
m.print()

print()
c1 = -0.0481
c2 = 64.1026
print(c1*v1+c2*v2)

Matrix 2x3:
 1,       0.0027,  0.125,  
-1,       0.0129,  0.875,  
[0, 0.0156, 1.0]
[0, 1, 64.1025641025641]
[1, 0.0, -0.04807692307692307]
Matrix 2x3:
 1,                    0.0,                 -0.04807692307692307, 
 0,                    1,                    64.1025641025641,    
Matrix 2x3:
 1,                    0.0,                 -0.04807692307692313, 
 0,                    1,                    64.1025641025641,    

[0.12497702, 0.8750235399999999]
